<a href="https://colab.research.google.com/github/andypedre2016/Data-MIning-Class/blob/main/bach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd

data = pd.read_csv('bach.csv')
data = data.set_index('choral_ID')
data

,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
choral_ID,,,,,,,,,,,,,,,,
000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M


In [15]:
#dividing features and labels
data_features = data.drop('chord_label', axis= 1)
data_labels = data[['chord_label']]

data_labels

,chord_label
choral_ID,
000106b_,F_M
000106b_,C_M
000106b_,C_M
000106b_,F_M
000106b_,F_M
...,...
015505b_,G_m
015505b_,G_m
015505b_,C_M


In [16]:
#One hot encodig 
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown= 'ignore')
data_Sparse = encoder.fit_transform(data_features)
data_Sparse

<5665x252 sparse matrix of type '<class 'numpy.float64'>'
	with 84975 stored elements in Compressed Sparse Row format>

In [17]:
#Diving into training and test
from sklearn.model_selection import train_test_split 
data_train_features, data_test_features, data_train_labels, data_test_labels = train_test_split(data_Sparse, data_labels, test_size = 0.2, random_state=42)
data_train_features

<4532x252 sparse matrix of type '<class 'numpy.float64'>'
	with 67980 stored elements in Compressed Sparse Row format>

In [18]:
#Creating classifier XGBoost
params = { "n_estimators": 400, 'tree_method':'gpu_hist', 'predictor':'gpu_predictor' }

from xgboost import XGBClassifier
model = XGBClassifier(**params)
model

XGBClassifier(n_estimators=400, predictor='gpu_predictor',
              tree_method='gpu_hist')

In [19]:
model.fit(data_train_features, data_train_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(n_estimators=400, objective='multi:softprob',
              predictor='gpu_predictor', tree_method='gpu_hist')

In [20]:
prediction = model.predict(data_test_features)

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(data_test_labels, prediction)

0.7263901147396293